## python gensim使用word2vec

In [8]:
import os

from gensim.models import word2vec
import jieba

In [9]:
# 读取训练数据
pos_file_list = os.listdir('data/pos')
neg_file_list = os.listdir('data/neg')
pos_file_list = [f'data/pos/{x}' for x in pos_file_list]
neg_file_list = [f'data/neg/{x}' for x in neg_file_list]
pos_neg_file_list = pos_file_list + neg_file_list

In [12]:
# 分词
for file in pos_neg_file_list:
    with open(file, 'r', encoding='utf-8') as f:
        text = f.read().strip()  # 去读文件，并去除空格
        text_cut = jieba.cut(text)  # 使用jieba进行分词
        
        result = ' '.join(text_cut)  # 把分词结果用空格组成字符串
        
        with open('test.txt', 'a', encoding='utf-8') as fw:
            fw.write(result)  # 把分好的词写入到新的文件里面
            pass
        pass
    pass


准备好分词后的文件，在一般的NLP处理中，会需要去停用词。由于word2vec的算法依赖于上下文文，而上下文有可能就是停用词。因此对于word2vec，我们可以不用去停用词

现在我们可以直接读分词后的文件到内存。这里使用了word2vec提供的LineSentence类来读文件，然后套用word2vec的模型


In [13]:
sentences = word2vec.LineSentence('test.txt')
model = word2vec.Word2Vec(sentences, hs=1, min_count=1, window=3, size=100)

训练好的模型一般都用于以下用途:
> 找出某一个词向量最相近的词集合

In [15]:
for val in model.wv.similar_by_word("酒店", topn=10):
    print(val[0], val[1])
    pass

宾馆 0.8037962913513184
饭店 0.7645680904388428
店 0.7105175852775574
城市 0.6644424796104431
如家 0.6424460411071777
酒店再也不会 0.6089881658554077
过程 0.6032401323318481
譬如 0.5995098352432251
汉庭 0.5930390954017639
行业 0.5919263362884521


> 查看两个词的相近程度

In [19]:
print(model.wv.similarity('酒店', '宾馆'))

0.8037963


> 找出不同类的词

In [20]:
word_list = ['宾馆', '酒店', '饭店', '服务']
print(model.wv.doesnt_match(word_list))

服务


c:\python\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


保存模型

In [21]:
model.save('model')  # 保存模型
# model = word2vec.Word2Vec.load('model')  # 加载模型

参数解释：
* 用gensim函数库训练Word2Vec模型有很多配置参数

> gensim.models.word2vec.Word2Vec(sentences=None,size=100,alpha=0.025,window=5, min_count=5, max_vocab_size=None, sample=0.001,seed=1, workers=3,min_alpha=0.0001, sg=0, hs=0, negative=5, cbow_mean=1, hashfxn=<built-in function hash>,iter=5,null_word=0, trim_rule=None, sorted_vocab=1, batch_words=10000)

* 参数:
    
> sentences：可以是一个list，对于大语料集，建议使用BrownCorpus,Text8Corpus或LineSentence构建。
    
> sg： 用于设置训练算法，默认为0，对应CBOW算法；sg=1则采用skip-gram算法。
    
> size：是指特征向量的维度，默认为100。大的size需要更多的训练数据,但是效果会更好. 推荐值为几十到几百。
    
> window：表示当前词与预测词在一个句子中的最大距离是多少
    
> alpha: 是学习速率
    
> seed：用于随机数发生器。与初始化词向量有关。
    
> min_count: 可以对字典做截断. 词频少于min_count次数的单词会被丢弃掉, 默认值为5
    
> max_vocab_size: 设置词向量构建期间的RAM限制。如果所有独立单词个数超过这个，则就消除掉其中最不频繁的一个。每一千万个单词需要大约1GB的RAM。设置成None则没有限制。
    
> sample: 高频词汇的随机降采样的配置阈值，默认为1e-3，范围是(0,1e-5)
    
> workers参数控制训练的并行数。
    
> hs: 如果为1则会采用hierarchical softmax技巧。如果设置为0（defaut），则negative sampling会被使用。
    
> negative: 如果>0,则会采用negativesamping，用于设置多少个noise words
    
> cbow_mean: 如果为0，则采用上下文词向量的和，如果为1（defaut）则采用均值。只有使用CBOW的时候才起作用。
    
> hashfxn： hash函数来初始化权重。默认使用python的hash函数
    
> iter： 迭代次数，默认为5
    
> trim_rule： 用于设置词汇表的整理规则，指定那些单词要留下，哪些要被删除。可以设置为None（min_count会被使用）或者一个接受()并返回RU·E_DISCARD,uti·s.RU·E_KEEP或者uti·s.RU·E_DEFAU·T的函数。
    
> sorted_vocab： 如果为1（defaut），则在分配word index 的时候会先对单词基于频率降序排序。
    
> batch_words：每一批的传递给线程的单词的数量，默认为10000
   